# Entraînement des Modèles CNN-LSTM et LSTM-CNN pour Détection d'Intrusions CAN

Ce notebook permet d'entraîner et d'évaluer deux architectures de deep learning pour la détection d'intrusions sur le bus CAN.

## 1. Imports et Configuration

In [1]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path

from canlock.attacks import AttackDatasetGenerator
from canlock.models import CNNLSTM, LSTMCNN
from canlock.training import create_dataloaders, IDSTrainer, calculate_metrics
from canlock.training.metrics import plot_confusion_matrix, plot_training_history, plot_class_distribution, print_metrics

# Configuration
WINDOW_SIZE = 50
BATCH_SIZE = 128
EPOCHS = 50
LR = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Device: {DEVICE}")
print(f"PyTorch version: {torch.__version__}")

Device: cpu
PyTorch version: 2.10.0+cpu


## 2. Génération du Dataset d'Attaques

Création d'un dataset synthétique avec attaques DoS, Fuzzing, Spoofing, Injection et traffic normal.

In [2]:
# Créer le générateur
dataset_dir = Path("../data/attack_datasets")

# Vérifier si le dataset existe déjà
if not (dataset_dir / "train.parquet").exists():
    print("Génération du dataset...")
    
    generator = AttackDatasetGenerator(seed=42)
    
    # Générer dataset avec IDs réels de la base de données
    df = generator.generate_from_database(num_samples=100000, use_real_ids=True)
    
    # Split et sauvegarde
    train_df, val_df, test_df = generator.split_dataset(df)
    
    dataset_dir.mkdir(parents=True, exist_ok=True)
    generator.save_dataset(train_df, dataset_dir / "train.parquet")
    generator.save_dataset(val_df, dataset_dir / "val.parquet")
    generator.save_dataset(test_df, dataset_dir / "test.parquet")
else:
    print("Dataset existant trouvé, chargement...")
    train_df = pd.read_parquet(dataset_dir / "train.parquet")
    val_df = pd.read_parquet(dataset_dir / "val.parquet")
    test_df = pd.read_parquet(dataset_dir / "test.parquet")

print(f"Train: {len(train_df)} samples")
print(f"Val: {len(val_df)} samples")
print(f"Test: {len(test_df)} samples")

Dataset existant trouvé, chargement...
Train: 70000 samples
Val: 15000 samples
Test: 15000 samples


### Visualisation de la Distribution des Classes

In [3]:
# Distribution des classes
fig = plot_class_distribution(train_df['label_numeric'].values, title="Distribution des Classes (Train)")
fig.show()

In [4]:
# Exemples de messages par classe
print("Exemples par classe:\n")
for label in train_df['label'].unique():
    example = train_df[train_df['label'] == label].iloc[0]
    print(f"{label.upper()}:")
    print(f"  CAN ID: 0x{example['can_identifier']:08X}")
    print(f"  DLC: {example['length']}")
    print(f"  Payload: {example['payload_bytes']}\n")

Exemples par classe:

NORMAL:
  CAN ID: 0x0CEEC5F9
  DLC: 8
  Payload: [185 167 125 158 165 155 163 190]

SPOOFING:
  CAN ID: 0x18FEFC00
  DLC: 8
  Payload: [193  48 183 174 171  71  92  92]

FUZZING:
  CAN ID: 0x18FEF500
  DLC: 8
  Payload: [241 227  76  19  52  88  89 182]

INJECTION:
  CAN ID: 0x1ADAC47B
  DLC: 8
  Payload: [145 169 176 155  77   8 115 102]

DOS:
  CAN ID: 0x0CF00300
  DLC: 8
  Payload: [255   0 138 255  58 255 255 102]



## 3. Création des DataLoaders

In [5]:
# Créer les dataloaders avec fenêtrage temporel
train_loader, val_loader, test_loader, scaler = create_dataloaders(
    train_df, 
    val_df, 
    test_df,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE,
    scaler_path=Path("checkpoints/scaler.pkl")
)

# Test un batch
sample_batch, sample_labels = next(iter(train_loader))
print(f"Batch shape: {sample_batch.shape}")  # (batch_size, window_size, input_dim)
print(f"Labels shape: {sample_labels.shape}")  # (batch_size,)
print(f"Input features: {sample_batch.shape[-1]}")

Creating datasets...
  Train dataset:
  Created 69951 windows from 70000 messages
  Validation dataset:
  Created 300 windows from 15000 messages
  Test dataset:
  Created 300 windows from 15000 messages
✓ Saved scaler to checkpoints\scaler.pkl

✓ Dataloaders created:
  Train batches: 547
  Val batches: 3
  Test batches: 3
Batch shape: torch.Size([128, 50, 12])
Labels shape: torch.Size([128])
Input features: 12


## 4. Entraînement CNN-LSTM

Architecture: CNN (extraction spatiale) → LSTM (modélisation temporelle)

In [6]:
# Créer le modèle CNN-LSTM
input_dim = 12  # CAN_ID + DLC + Priority + TimeDelta + 8 payload bytes

cnn_lstm_model = CNNLSTM(
    input_dim=input_dim,
    num_classes=5,
    window_size=WINDOW_SIZE
)

print(f"Modèle CNN-LSTM:")
print(f"  Paramètres: {cnn_lstm_model.count_parameters():,}")
print(cnn_lstm_model)

Modèle CNN-LSTM:
  Paramètres: 705,221
CNNLSTM(
  (cnn): Sequential(
    (0): Conv1d(12, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
    (5): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.3, inplace=False)
  )
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=64, out_features=5, bias=True)
  )
)


In [7]:
# Créer le trainer
cnn_lstm_trainer = IDSTrainer(
    model=cnn_lstm_model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=DEVICE,
    lr=LR,
    checkpoint_dir=Path("checkpoints"),
    early_stopping_patience=10
)

# Entraîner
cnn_lstm_history = cnn_lstm_trainer.train(num_epochs=EPOCHS, save_best=True)


Starting training on cpu
Model: CNNLSTM
Parameters: 705,221
Epochs: 50
Batch size: 128
------------------------------------------------------------

[Epoch 1/50]


Validation: 100%|██████████| 3/3 [00:00<00:00, 40.03it/s]



Epoch Summary:
  Train Loss: 1.1457 | Train Acc: 0.5558
  Val Loss:   0.7918 | Val Acc:   0.7067
  Val F1 (macro): 0.4509
  Learning Rate: 0.001000
✓ Checkpoint saved to checkpoints\best_model_cnnlstm.pt
  ✓ Saved best model (val_loss: 0.7918)

[Epoch 2/50]


Validation: 100%|██████████| 3/3 [00:00<00:00, 26.72it/s]



Epoch Summary:
  Train Loss: 0.9114 | Train Acc: 0.6348
  Val Loss:   0.7394 | Val Acc:   0.7100
  Val F1 (macro): 0.4558
  Learning Rate: 0.001000
✓ Checkpoint saved to checkpoints\best_model_cnnlstm.pt
  ✓ Saved best model (val_loss: 0.7394)

[Epoch 3/50]


...


Epoch Summary:
  Train Loss: 0.5227 | Train Acc: 0.7964
  Val Loss:   0.4437 | Val Acc:   0.8100
  Val F1 (macro): 0.7079
  Learning Rate: 0.000500
  No improvement for 8 epoch(s)

[Epoch 32/50]


Validation: 100%|██████████| 3/3 [00:00<00:00, 20.87it/s]



Epoch Summary:
  Train Loss: 0.5205 | Train Acc: 0.7950
  Val Loss:   0.4268 | Val Acc:   0.8233
  Val F1 (macro): 0.7318
  Learning Rate: 0.000500
  No improvement for 9 epoch(s)

[Epoch 33/50]


Validation: 100%|██████████| 3/3 [00:00<00:00, 33.53it/s]



Epoch Summary:
  Train Loss: 0.5191 | Train Acc: 0.7957
  Val Loss:   0.4369 | Val Acc:   0.8200
  Val F1 (macro): 0.7252
  Learning Rate: 0.000500
  No improvement for 10 epoch(s)

Early stopping triggered after 33 epochs

Training Complete!
  Total time: 42.54 minutes
  Best val loss: 0.4158



In [8]:
# Visualiser l'historique d'entraînement
fig = plot_training_history(cnn_lstm_history, title="CNN-LSTM Training History")
fig.show()

## 5. Entraînement LSTM-CNN

Architecture: LSTM (modélisation temporelle) → CNN (extraction de patterns)

In [9]:
# Créer le modèle LSTM-CNN
lstm_cnn_model = LSTMCNN(
    input_dim=input_dim,
    num_classes=5,
    window_size=WINDOW_SIZE
)

print(f"Modèle LSTM-CNN:")
print(f"  Paramètres: {lstm_cnn_model.count_parameters():,}")
print(lstm_cnn_model)

Modèle LSTM-CNN:
  Paramètres: 623,557
LSTMCNN(
  (lstm): LSTM(12, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (lstm_dropout): Dropout(p=0.3, inplace=False)
  (cnn): Sequential(
    (0): Conv1d(256, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
    (5): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.3, inplace=False)
  )
  (global_avg_pool): AdaptiveAvgPool1d(output_size=1)
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
   

In [10]:
# Créer le trainer
lstm_cnn_trainer = IDSTrainer(
    model=lstm_cnn_model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=DEVICE,
    lr=LR,
    checkpoint_dir=Path("checkpoints"),
    early_stopping_patience=10
)

# Entraîner
lstm_cnn_history = lstm_cnn_trainer.train(num_epochs=EPOCHS, save_best=True)


Starting training on cpu
Model: LSTMCNN
Parameters: 623,557
Epochs: 50
Batch size: 128
------------------------------------------------------------

[Epoch 1/50]


Validation: 100%|██████████| 3/3 [00:00<00:00,  9.14it/s]



Epoch Summary:
  Train Loss: 1.1481 | Train Acc: 0.5453
  Val Loss:   0.8203 | Val Acc:   0.6800
  Val F1 (macro): 0.4187
  Learning Rate: 0.001000
✓ Checkpoint saved to checkpoints\best_model_lstmcnn.pt
  ✓ Saved best model (val_loss: 0.8203)

[Epoch 2/50]


Validation: 100%|██████████| 3/3 [00:00<00:00,  8.72it/s]



Epoch Summary:
  Train Loss: 0.8199 | Train Acc: 0.6795
  Val Loss:   0.6542 | Val Acc:   0.7667
  Val F1 (macro): 0.5547
  Learning Rate: 0.001000
✓ Checkpoint saved to checkpoints\best_model_lstmcnn.pt
  ✓ Saved best model (val_loss: 0.6542)

[Epoch 3/50]


...


Epoch Summary:
  Train Loss: 0.3152 | Train Acc: 0.8738
  Val Loss:   0.4010 | Val Acc:   0.8333
  Val F1 (macro): 0.7523
  Learning Rate: 0.000250
  No improvement for 8 epoch(s)

[Epoch 38/50]


Validation: 100%|██████████| 3/3 [00:00<00:00, 12.24it/s]



Epoch Summary:
  Train Loss: 0.3139 | Train Acc: 0.8748
  Val Loss:   0.4075 | Val Acc:   0.8367
  Val F1 (macro): 0.7606
  Learning Rate: 0.000250
  No improvement for 9 epoch(s)

[Epoch 39/50]


Validation: 100%|██████████| 3/3 [00:00<00:00, 11.11it/s]


Epoch Summary:
  Train Loss: 0.3091 | Train Acc: 0.8759
  Val Loss:   0.3793 | Val Acc:   0.8400
  Val F1 (macro): 0.7581
  Learning Rate: 0.000250
  No improvement for 10 epoch(s)

Early stopping triggered after 39 epochs

Training Complete!
  Total time: 140.34 minutes
  Best val loss: 0.3742



In [11]:
# Visualiser l'historique d'entraînement
fig = plot_training_history(lstm_cnn_history, title="LSTM-CNN Training History")
fig.show()

## 6. Évaluation sur le Test Set

Comparaison des performances des deux architectures

In [12]:
# Évaluer CNN-LSTM
print("=" * 60)
print("CNN-LSTM TEST RESULTS")
print("=" * 60)
cnn_lstm_metrics = cnn_lstm_trainer.test(test_loader)

CNN-LSTM TEST RESULTS

Evaluating on test set...


Testing: 100%|██████████| 3/3 [00:00<00:00, 19.97it/s]


EVALUATION METRICS

                    Overall Performance                     
------------------------------------------------------------
  Accuracy:      0.8333
  F1 (macro):    0.7537
  F1 (micro):    0.8333
  ROC-AUC:       0.9649

                   Per-Class Performance                    
------------------------------------------------------------
Class            Precision     Recall   F1-Score    Support
------------------------------------------------------------
Normal              0.9595     0.9861     0.9726        144
DoS                 0.8846     0.5610     0.6866         41
Fuzzing             0.5833     0.8537     0.6931         41
Spoofing            0.7381     1.0000     0.8493         31
Injection           0.7917     0.4419     0.5672         43



In [13]:
# Évaluer LSTM-CNN
print("=" * 60)
print("LSTM-CNN TEST RESULTS")
print("=" * 60)
lstm_cnn_metrics = lstm_cnn_trainer.test(test_loader)

LSTM-CNN TEST RESULTS

Evaluating on test set...


Testing: 100%|██████████| 3/3 [00:00<00:00, 11.61it/s]



EVALUATION METRICS

                    Overall Performance                     
------------------------------------------------------------
  Accuracy:      0.8300
  F1 (macro):    0.7580
  F1 (micro):    0.8300
  ROC-AUC:       0.9634

                   Per-Class Performance                    
------------------------------------------------------------
Class            Precision     Recall   F1-Score    Support
------------------------------------------------------------
Normal              0.9524     0.9722     0.9622        144
DoS                 0.8250     0.8049     0.8148         41
Fuzzing             0.6364     0.6829     0.6588         41
Spoofing            0.7297     0.8710     0.7941         31
Injection           0.6562     0.4884     0.5600         43



### Matrices de Confusion

In [14]:
# Obtenir les prédictions pour les matrices de confusion
@torch.no_grad()
def get_predictions(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    for data, labels in dataloader:
        data = data.to(device)
        output = model(data)
        _, preds = output.max(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())
    
    return np.array(all_labels), np.array(all_preds)

# CNN-LSTM
y_true, y_pred_cnn_lstm = get_predictions(cnn_lstm_model, test_loader, DEVICE)
fig = plot_confusion_matrix(y_true, y_pred_cnn_lstm, title="CNN-LSTM Confusion Matrix")
fig.show()

# LSTM-CNN
_, y_pred_lstm_cnn = get_predictions(lstm_cnn_model, test_loader, DEVICE)
fig = plot_confusion_matrix(y_true, y_pred_lstm_cnn, title="LSTM-CNN Confusion Matrix")
fig.show()

### Comparaison des Modèles

In [15]:
# Tableau comparatif
comparison = pd.DataFrame({
    'Métrique': ['Accuracy', 'F1 (macro)', 'F1 (micro)', 'Paramètres'],
    'CNN-LSTM': [
        f"{cnn_lstm_metrics['accuracy']:.4f}",
        f"{cnn_lstm_metrics['f1_macro']:.4f}",
        f"{cnn_lstm_metrics['f1_micro']:.4f}",
        f"{cnn_lstm_model.count_parameters():,}"
    ],
    'LSTM-CNN': [
        f"{lstm_cnn_metrics['accuracy']:.4f}",
        f"{lstm_cnn_metrics['f1_macro']:.4f}",
        f"{lstm_cnn_metrics['f1_micro']:.4f}",
        f"{lstm_cnn_model.count_parameters():,}"
    ]
})

print("\n" + "="*60)
print("COMPARAISON DES MODÈLES")
print("="*60)
print(comparison.to_string(index=False))


COMPARAISON DES MODÈLES
  Métrique CNN-LSTM LSTM-CNN
  Accuracy   0.8333   0.8300
F1 (macro)   0.7537   0.7580
F1 (micro)   0.8333   0.8300
Paramètres  705,221  623,557
